## FILE-FRIEND-URL-to-COS-Loader

In [13]:
#imports.... Run this each time after restarting the Kernel
import json
from botocore.client import Config
import ibm_boto3
import requests


## Hardwire endpoints

In [14]:

service_endpoint = 'https://s3-api.us-geo.objectstorage.softlayer.net'
auth_endpoint = 'https://iam.bluemix.net/oidc/token'



## Pull Creds from: CONSOLE - Dashboard > COS > Service Credentials.  Take ADMIN set.  

In [15]:

cos_credentials = {
  "apikey": "########rl3Gqgd-2IMZUHGRiHZbSH9G9WrzU",
  "cos_hmac_keys": {
    "access_key_id": "#######2c4abcaf68187e036c53c5",
    "secret_access_key": "#######cf82729381b8f5066277ead12d98f05b7cd58b"
  },
  "endpoints": "https://cos-service.bluemix.net/endpoints",
  "iam_apikey_description": "Auto generated apikey during resource-key operation for Instance - crn:v1:bluemix:public:cloud-object-storage:global:a/6bd61f31ad3927791bc9b0bebfdcb749:ee080c49-f45a-468d-89da-6a7c060d055b::",
  "iam_apikey_name": "auto-generated-apikey-######-a92c-4abc-af68-187e036c53c5",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::role:Administrator",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/6bd61f31ad3927791bc9b0bebfdcb749::serviceid:ServiceId-3f31e855-2dce-4377-a143-8291529c49fb",
  "resource_instance_id": "crn:v1:bluemix:public:cloud-object-storage:global:a/6bd61f31ad3927791bc9b0bebfdcb749:ee080c49-f45a-468d-89da-6a7c060d055b::"
}


# COS and effect

In [16]:
cos = ibm_boto3.resource('s3',
                        ibm_api_key_id=cos_credentials['apikey'],
                        ibm_service_instance_id=cos_credentials['resource_instance_id'],
                        ibm_auth_endpoint= auth_endpoint,
                        config=Config(signature_version='oauth'),
                        endpoint_url= service_endpoint )


# Bucket Time

In [17]:
from uuid import uuid4

bucket_uid = str(uuid4())
buckets = ['audio-data-' + bucket_uid, 'outputs-' + bucket_uid]

for bucket in buckets:
   if not cos.Bucket(bucket) in cos.buckets.all():
       print('Creating bucket “{}“...'.format(bucket))
       try:
           cos.create_bucket(Bucket=bucket)
       except ibm_boto3.exceptions.ibm_botocore.client.ClientError as e:
           print('Error: {}.'.format(e.response['Error']['Message']))

data_links = ['http://github.com/mamoonraja/call-center-think18/blob/master/resources/audio_samples/sample1-addresschange-positive.ogg',
              'http://github.com/mamoonraja/call-center-think18/blob/master/resources/audio_samples/sample2-address-negative.ogg',
              'http://github.com/mamoonraja/call-center-think18/blob/master/resources/audio_samples/sample3-shirt-return-weather-chitchat.ogg',
              'http://github.com/mamoonraja/call-center-think18/blob/master/resources/audio_samples/sample4-angryblender-sportschitchat-recovery.ogg',
              'http://github.com/mamoonraja/call-center-think18/blob/master/resources/audio_samples/sample5-calibration-toneandcontext.ogg',
              'http://github.com/mamoonraja/call-center-think18/blob/master/resources/audio_samples/jfk_1961_0525_speech_to_put_man_on_moon.ogg']



Creating bucket “audio-data-ef54588d-bca4-4444-b490-a1e11eea8e46“...
Creating bucket “outputs-ef54588d-bca4-4444-b490-a1e11eea8e46“...


# Upload the Files 

In [18]:
import urllib
from urllib.request import urlopen

bucket_obj = cos.Bucket(buckets[0])

for data_link in data_links:
   filename=data_link.split('/')[-1]
   print('Uploading data {}...'.format(filename))
   with urlopen(data_link) as data:
       bucket_obj.upload_fileobj(data, filename)
       print('{} is uploaded.'.format(filename))


Uploading data sample1-addresschange-positive.ogg...
sample1-addresschange-positive.ogg is uploaded.
Uploading data sample2-address-negative.ogg...
sample2-address-negative.ogg is uploaded.
Uploading data sample3-shirt-return-weather-chitchat.ogg...
sample3-shirt-return-weather-chitchat.ogg is uploaded.
Uploading data sample4-angryblender-sportschitchat-recovery.ogg...
sample4-angryblender-sportschitchat-recovery.ogg is uploaded.
Uploading data sample5-calibration-toneandcontext.ogg...
sample5-calibration-toneandcontext.ogg is uploaded.
Uploading data jfk_1961_0525_speech_to_put_man_on_moon.ogg...
jfk_1961_0525_speech_to_put_man_on_moon.ogg is uploaded.
